In [1]:
import pandas as pd
import re
from sklearn import model_selection

In [2]:
b1 = pd.read_csv("Coded Tweets (Aggregated)/Tweets Batch 1 (N=1000)_ALLRATERS.csv")
b2 = pd.read_csv("Coded Tweets (Aggregated)/Tweets Batch 2 (N=2000)_ALLRATERS.csv")
b3 = pd.read_csv("Coded Tweets (Aggregated)/Tweets Batch 3 (N=2000)_ALLRATERS.csv")

In [3]:
b = pd.concat([b1, b2, b3], axis = 0)

In [4]:
b.shape

(5001, 64)

In [5]:
len(b['Tweet'].unique()) #duplicates due to retweeting

4282

In [6]:
set(b1['Tweet']) & set(b2['Tweet'])

{"RT @AnnCoulter: Trump's right: He can win Michigan. They lost their industry to trade deals &amp; now lose everything else to refugees. https:/… ",
 'RT @BernardKeane: On the day the Coalition says it wants to fight an election on trade unions, the head of the ASX is forced to resign over… ',
 'RT @BernieSanders: We need policies that bring us together and create a middle class that works for all of us rather than an economy that w… ',
 "RT @C_XXIII: Uhh my mans can take me to Jack in the box for 2 for $1 tacos &amp; I still feel spoiled?? money ain't everything, boo. https://t.c… ",
 'RT @Chiba_Lotte: 【3月15日オープン戦 M2-0H】 石川投手が7回79球2安打無失点の好投！8回に岡田選手が決勝打！ ▼本日の成績はこちら https://t.co/alb2r7rVgn #chibalotte https://t.co/5Jkht8xbIQ ',
 'RT @Chiba_Lotte: 【3月19日オープン戦 M3-3D】 清田選手がホームランを含む3安打2打点の活躍！ ▼本日の成績はこちら https://t.co/LhReylm3a8 #chibalotte https://t.co/4Rd9jHNXml ',
 "RT @Chris_1791: All Marines Forced To Go Through Training To Erase 'Unconscious Bias' As Women Enter Combat Roles: Marine le

In [7]:
b.drop_duplicates(inplace=False).shape #some of them are complete duplicates

(4840, 64)

In [8]:
b.loc[b.duplicated(), 'Tweet']

77      RT @homeIwt: Childish Gambino // African-Ameri...
151     RT @scrowder: FACT: Hitler was a left-wing soc...
165     RT @homeIwt: Michael Jackson // African-Americ...
222     RT @AnnCoulter: Trump's right: He can win Mich...
373     RT @dovorah001: #UtahForTrump #Utah There is a...
449     RT @OttomanArchive: Muslims, Christians and Je...
488     RT @Chiba_Lotte: 【3月19日オープン戦 M3-3D】 清田選手がホームラン...
490     RT @C_XXIII: Uhh my mans can take me to Jack i...
588     RT @xonecole: Meet Akilah Johnson: The 1st Afr...
608     RT @TRobinsonNewEra: The so called left wing h...
614     RT @ggreenwald: Can't say it enough: the abili...
639     RT @Nigel_Farage: When we Leave the EU we can ...
668     RT @ColeMcNeely: Bernie's Immigration Plan: Ha...
679     RT @historyepics: The only known photograph of...
714     RT @BernardKeane: On the day the Coalition say...
715     RT @SadHappyAmazing: The only known photograph...
730     RT @loudobbsnews: Left wing Activists Try to S...
733     RT @Di

In [9]:
b1 = b1.rename(columns = {'hatespeech7namecalaling1':'hatespeech7', 'nanmecalling2':'namecalling2'})
b2 = b2.rename(columns = {'hatespeech7namecalaling1':'hatespeech7', 'nanmecalling2':'namecalling2'})
b3 = b3.rename(columns = {'hatespeech7namecalaling1':'hatespeech7', 'nanmecalling2':'namecalling2'})

In [10]:
r1 = pd.read_csv("Coded Tweets (Aggregated)/rater_key_1.csv")
r2 = pd.read_csv("Coded Tweets (Aggregated)/rater_key_2.csv")
r3 = pd.read_csv("Coded Tweets (Aggregated)/rater_key_3.csv")

In [11]:
b1.columns

Index(['Tweet', 'Threat1', 'Threat2', 'Threat3', 'Threat4', 'Threat5',
       'Threat6', 'Threat7', 'Obscenity1', 'Obscenity2', 'Obscenity3',
       'Obscenity4', 'Obscenity5', 'Obscenity6', 'Obscenity7', 'namecalling1',
       'namecalling2', 'namecalling3', 'namecalling4', 'namecalling5',
       'namecalling6', 'namecalling7', 'hatespeech1', 'hatespeech2',
       'hatespeech3', 'hatespeech4', 'hatespeech5', 'hatespeech6',
       'hatespeech7', 'CAPS1', 'CAPS2', 'CAPS3', 'CAPS4', 'CAPS5', 'CAPS6',
       'CAPS7', 'stereotypes1', 'stereotypes2', 'stereotypes3', 'stereotypes4',
       'stereotypes5', 'stereotypes6', 'stereotypes7', 'negprejudice1',
       'negprejudice2', 'negprejudice3', 'negprejudice4', 'negprejudice5',
       'negprejudice6', 'negprejudice7', 'porn1', 'porn2', 'porn3', 'porn4',
       'porn5', 'porn6', 'porn7', 'noneng1', 'noneng2', 'noneng3', 'noneng4',
       'noneng5', 'noneng6', 'noneng7'],
      dtype='object')

In [12]:
#make rater tables consistent

r1['Rater'] = r1['Rater'].str.strip()

r2.columns = r1.columns

r3.columns = r1.columns

In [13]:
#merge rater tables

r = r1.merge(r2, how = 'left', on = "Rater")

r = r.merge(r3, how = 'left', on = "Rater")

r .columns = ['Number_1', 'Rater', "Number_2", 'Number_3']

r.loc[r['Rater'] == "Parker", 'Number_3'] = 7

In [14]:
#dictionary of correspondences for rater tables

dict2_name = r[["Number_2", "Rater"]].set_index("Number_2")['Rater'].to_dict()

dict3_name = r[["Number_3", "Rater"]].set_index("Number_3")['Rater'].to_dict()

In [15]:
#convert rater numbers to rater names

for i in range(1,8):
    b2.columns = [re.sub(str(i), dict2_name[i], x) for x in b2.columns.values]

for i in range(1,8):
    b3.columns = [re.sub(str(i), dict3_name[i], x) for x in b3.columns.values]

In [16]:
dict_name_1 = r[["Rater", "Number_1"]].set_index("Rater")["Number_1"]

In [17]:
#convert rater names to spreadsheet 1 rater numbers

for i in r['Rater'].tolist():
    b2.columns = [re.sub(i, str(dict_name_1[i]), x) for x in b2.columns.values]

for i in r['Rater'].tolist():
    b3.columns = [re.sub(i, str(dict_name_1[i]), x) for x in b3.columns.values]

In [18]:
data = b1.append(b2)

In [19]:
data = data.append(b3)

In [20]:
data['ID'] = range(0,data.shape[0])

In [32]:
data.columns

Index(['CAPS1', 'CAPS2', 'CAPS3', 'CAPS4', 'CAPS5', 'CAPS6', 'CAPS7',
       'Obscenity1', 'Obscenity2', 'Obscenity3', 'Obscenity4', 'Obscenity5',
       'Obscenity6', 'Obscenity7', 'Threat1', 'Threat2', 'Threat3', 'Threat4',
       'Threat5', 'Threat6', 'Threat7', 'Tweet', 'hatespeech1', 'hatespeech2',
       'hatespeech3', 'hatespeech4', 'hatespeech5', 'hatespeech6',
       'hatespeech7', 'namecalling1', 'namecalling2', 'namecalling3',
       'namecalling4', 'namecalling5', 'namecalling6', 'namecalling7',
       'negprejudice1', 'negprejudice2', 'negprejudice3', 'negprejudice4',
       'negprejudice5', 'negprejudice6', 'negprejudice7', 'noneng1', 'noneng2',
       'noneng3', 'noneng4', 'noneng5', 'noneng6', 'noneng7', 'porn1', 'porn2',
       'porn3', 'porn4', 'porn5', 'porn6', 'porn7', 'stereotypes1',
       'stereotypes2', 'stereotypes3', 'stereotypes4', 'stereotypes5',
       'stereotypes6', 'stereotypes7', 'ID'],
      dtype='object')

In [33]:
data.to_csv('combined_tweets.csv')

In [22]:
train, test = model_selection.train_test_split(data, test_size = 0.2, random_state = 123)

In [23]:
train.isnull().sum()

CAPS1            1988
CAPS2            2779
CAPS3            1991
CAPS4            1991
CAPS5            2021
CAPS6            2004
CAPS7            2011
Obscenity1       1988
Obscenity2       2779
Obscenity3       1990
Obscenity4       1990
Obscenity5       2032
Obscenity6       2004
Obscenity7       2011
Threat1          1988
Threat2          2779
Threat3          1991
Threat4          1990
Threat5          2034
Threat6          2004
Threat7          2010
Tweet               1
hatespeech1      1988
hatespeech2      2779
hatespeech3      1990
hatespeech4      1990
hatespeech5      2024
hatespeech6      2004
hatespeech7      2011
namecalling1     1988
                 ... 
namecalling7     2011
negprejudice1    1988
negprejudice2    2779
negprejudice3    1991
negprejudice4    1990
negprejudice5    2020
negprejudice6    2004
negprejudice7    2011
noneng1          1988
noneng2          2779
noneng3          1990
noneng4          1991
noneng5          2020
noneng6          2004
noneng7   

In [24]:
train = train[train['Tweet'].isnull() == False]

In [25]:
train.to_csv("train.csv")

In [26]:
test.to_csv("test.csv")

In [27]:
train.describe()

,CAPS1,CAPS2,CAPS3,CAPS4,CAPS5,CAPS6,CAPS7,Obscenity1,Obscenity2,Obscenity3,...,porn6,porn7,stereotypes1,stereotypes2,stereotypes3,stereotypes4,stereotypes5,stereotypes6,stereotypes7,ID
count,2012.000000,1221.000000,2009.000000,2009.000000,1979.000000,1996.000000,1989.000000,2012.000000,1221.000000,2010.000000,...,1996.000000,1989.000000,2012.000000,1221.000000,2009.000000,2010.000000,1980.000000,1996.000000,1989.000000,3999.000000
mean,0.013917,0.013923,0.020906,0.024390,0.013138,0.014529,0.004525,0.044235,0.100737,0.075622,...,0.010020,0.003017,0.172962,0.185094,0.051767,0.060199,0.046970,0.217435,0.055807,2488.707177
std,0.117174,0.117220,0.143105,0.154296,0.113894,0.119688,0.067132,0.205667,0.301104,0.264458,...,0.099622,0.054854,0.378309,0.388533,0.221612,0.237914,0.211627,0.412604,0.229606,1442.376368
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1239.500000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2486.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3739.500000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5000.000000
